In [20]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train = pd.read_csv("adult.csv")

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
ada_model = AdaBoostClassifier(n_estimators=50, algorithm='SAMME', random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')

In [22]:
base_models = [
    ('RandomForest', rf_model),
    ('AdaBoost', ada_model),
    ('GradientBoosting', gb_model),
    ('XGBoost', xgb_model)
]

stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    cv=5  
)

In [24]:
train['sex'] = train['sex'].map({'Male': 0, 'Female': 1})

# میانگین برای سن
train['age'] = train['age'].fillna(train['age'].median())

# حذف ستون‌ های غیرضروری
train = train.drop(['marital.status', 'relationship', 'race', 'native.country', 'fnlwgt'], axis=1)

# features و target
X = train.drop('income', axis=1)
y = train['income'].map({'<=50K': 0, '>50K': 1})

# جایگزینی مقادیر ?
X['workclass'] = X['workclass'].replace('?', 'Unknown')
X['occupation'] = X['occupation'].replace('?', 'Unknown')

# تبدیل ستون های کتگوریکال از ابجکت به عدد
categorical_cols = ['workclass', 'education', 'occupation']
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
stacking_model.fit(X_train, y_train)

y_pred_stack = stacking_model.predict(X_val)

from sklearn.metrics import accuracy_score
acc_stack = accuracy_score(y_val, y_pred_stack)
print("Stacking Model Accuracy:", acc_stack)

Stacking Model Accuracy: 0.8569015814524796


-------------------------------

تو این مرحله تصمیم گرفتم پیش بینی های تمام مدل هایی که قبلا آموزش داده بودم یعنی 
Random Forest - AdaBoost - Gradient Boosting و XGBoost
رو با هم ترکیب کنم تا یک مدل نهایی بسازم
ایده اش این بود که هر مدل نقاط قوت خودش رو داره و ممکنه بعضی نمونه ها رو درست پیش بینی کنه که بقیه مدل ها اشتباه گرفتن پس با استیکینگ پیش بینی های همه مدل ها رو به عنوان ورودی به یک مدل نهایی میدم
این کار باعث شد مدل نهایی از تجربه همه مدل ها استفاده کنه و پیش بینی ها دقیق تر بشه بعد از آموزش استیکینگ دقت مدل نهایی روی داده های ولیدیشن رو حساب کردم و دیدم که عملکرد بهتر از اکثر مدل های منفرد بود